<a href="https://colab.research.google.com/github/BeaverWorksMedlytics2020/Week2/blob/master/Notebooks/05_NeuralNetworks/NeuralNetworks_Tutorial_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a Simple Neural Network with Tensorflow Keras

In this notebook we are going to walk through building a simple neural network to classify sequence data. This tutorial will be meant as a fast overview of building/training neural networks with Keras.

In [ ]:
# Import useful libraries

#Needed for terminal functions (i.e. wget)
import os

#For plotting
import matplotlib.pyplot as plt

#For dataframe manipulation
import pandas as pd

#For data preprocessing
from sklearn.preprocessing import StandardScaler #Use StandardScaler from scikitlearn
from sklearn.utils import shuffle #Used to shuffle up examples before training

#Keras-related imports
from keras.models import Sequential  #we will build our models layer by layer
from keras.layers import Dense  #we want to use dense layers in our model

#Keras is built on top of tensorflow library
import tensorflow as tf #tf has many helpful functions for training networks like loss functions, optimization methods, etc.

2021-07-14 14:11:44.680961: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-14 14:11:44.681041: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#Load the dataframe that contains all features calculated in the last notebook (only run once)
os.system('wget https://raw.githubusercontent.com/BeaverWorksMedlytics2020/Data_Public/master/NotebookExampleData/Week2/spoken_digit_manual_features.csv')

--2021-07-14 14:11:46--  https://raw.githubusercontent.com/BeaverWorksMedlytics2020/Data_Public/master/NotebookExampleData/Week2/spoken_digit_manual_features.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220478 (215K) [text/plain]
Saving to: ‘spoken_digit_manual_features.csv.2’

     0K .......... .......... .......... .......... .......... 23% 32.6M 0s
    50K .......... .......... .......... .......... .......... 46% 33.6M 0s
   100K .......... .......... .......... .......... .......... 69% 38.3M 0s
   150K .......... .......... .......... .......... .......... 92% 55.0M 0s
   200K .......... .....                                      100% 50.8M=0.005s

2021-07-14 14:11:46 (38.9 MB/s) - ‘spoken_digit_manual_features.csv.2’ saved [220478/220478]



0

## Load Training Data

In [ ]:
#Load dataframe and print its contents to jog memory
spoken_df = pd.read_csv('spoken_digit_manual_features.csv', index_col = 0)
print(spoken_df.head(10))
print('\n')

#Check how many unique speakers exist in the dataset
speakers=set(spoken_df['speaker'])
print(f'There are {len(speakers)} unique speakers in the dataset')

# Our goal for this is to build a neural network that learns to classify which
# of 5 speakers is recorded in a sample based on the features:
# spectral centroid, spectral flatness, and maximum frequency


                file  digit   speaker  trial           SC        SF  \
0   5_yweweler_8.wav      5  yweweler      8  1029.497959  0.397336   
1    3_george_49.wav      3    george      4  1881.296834  0.387050   
2  9_yweweler_44.wav      9  yweweler      4  1093.951856  0.394981   
3  8_yweweler_33.wav      8  yweweler      3  1409.543285  0.487496   
4      7_theo_34.wav      7      theo      3   887.361601  0.396825   
5   1_jackson_45.wav      1   jackson      4  1007.568129  0.324100   
6  6_yweweler_18.wav      6  yweweler      1  1286.701352  0.498813   
7    9_george_35.wav      9    george      3  1405.092061  0.353083   
8   9_jackson_32.wav      9   jackson      3  1172.899961  0.477907   
9    8_george_26.wav      8    george      2  1959.977577  0.462901   

           MF  
0  745.878340  
1  323.943662  
2  244.648318  
3  392.350401  
4  130.640309  
5  216.306156  
6  400.715564  
7  447.239693  
8  114.892780  
9  320.537966  


There are 5 unique speakers in the datas

## Structure Neural Network

In [ ]:
# Build the keras neural network

#this allows us to add layers sequentially (i.e. first->last)
model = tf.keras.Sequential()

#create a first layer of 12 neurons, and a rectified linear unit activation function
model.add(tf.keras.layers.Dense(8, input_shape=(3,), activation=tf.nn.relu)) #input dimension needs to be number of features

#add two dense layers with 8 units each
#(note that we don't need to specify input size because keras determines input size from previous layer)
model.add(tf.keras.layers.Dense(8, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(8, activation=tf.nn.relu))

# output dimension needs to be number of classes in order for each to get a score
model.add(tf.keras.layers.Dense(5, activation=tf.nn.softmax)) 

2021-07-14 14:11:51.440786: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-14 14:11:51.441023: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-14 14:11:51.441036: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-14 14:11:51.441056: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-3d82bae0-f509-4ec0-92dc-a9b1bbc77ba9): /proc/driver/nvidia/version does not exist
2021-07-14 14:11:51.441297: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operatio

## Specify a Loss Function and an Optimizer for NN Model

Let's describe why each of these components is necessary, and how it is used in training a neural network.

**Loss Function** - This is the quantity that should be minimized when the network is trained. (It is like the mean squared error for a linear regression.) A neural network can use squared error as a loss function, but there are also other options. In the case of a neural network trying to classify samples into 1 of n categories system a common choice is called cross entropy loss.

**Optimizer** - When a neural network is trained, it changes weights in the network to minimize the loss function. The optimizer governs how the neural network iteratively changes its weights as it minimizes loss. Many optimizers use the derivative of the loss function with respect to all the weights to decide which direction to change network weights.

In [ ]:
#Specify a loss function for our network

#Note that the metrics input argument governs what will be reported as the network is trained 
model.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) , metrics = ['accuracy'])


## Convert Labels into "Onehot" Vectors

Predictions output by the model need to be compared to some truth label. Currently, the model predicts a 5-element vector of "prediction values" for every sample. The truth labels thus need to be converted to a 5-element vector with a 1 in the correct index and zeros in all others

In [ ]:
#make dictionary to convert from speaker names to indices
name2int_dict = {name: ind for (ind, name) in enumerate(set(spoken_df['speaker']))}

y_labels = spoken_df['speaker']
#set y_labels to be indices of speaker
y_labels = [name2int_dict[name] for name in y_labels]


## Standardize Data and split into train/validation/test sets

Scaling data is generally good practice before attempting to fit a model. Having inputs with large differences in scale can affect how the optimizer changes weights to minimize the loss function

In [ ]:
#downselect to only the 3 columns of the dataset we are learning from
X_data = spoken_df[['SC', 'SF', 'MF']].to_numpy()

#Decide how large to make validation and test sets
n_val = 250
n_test = 250

#Shuffle data before partitioning
X_data, y_labels = shuffle(X_data, y_labels, random_state = 25)

#Partition
X_data_test, y_labels_test = X_data[:n_test,:], y_labels[:n_test]
X_data_val, y_labels_val = X_data[n_test:n_test+n_val,:], y_labels[n_test:n_test+n_val]
X_data_train, y_labels_train = X_data[n_test+n_val:,:], y_labels[n_test+n_val:]

#Scale data
scaler = StandardScaler()
X_data_train=scaler.fit_transform(X_data_train)
X_data_val = scaler.transform(X_data_val)
X_data_test = scaler.transform(X_data_test)

#convert labels to onehot
y_labels_train = tf.keras.utils.to_categorical(y_labels_train, 5)
y_labels_val =  tf.keras.utils.to_categorical(y_labels_val, 5)
y_labels_test =  tf.keras.utils.to_categorical(y_labels_test, 5)

training_set = tf.data.Dataset.from_tensor_slices((X_data_train, y_labels_train))

## Fit Model to Data, Specify Number of Epochs and Batch Size

**Batch Size** - In each iteration of the optimizer, how many samples are taken into account when calculating derivatives of the loss function? (If batch size is less than number of samples, there will be multiple optimization iterations per epoch.)

**Epochs** - How many times should the data be passed through before optimization is finished?

In [ ]:
epochs = 50
batch_size = 100

training_set = training_set.batch(batch_size) #set batch size

for epoch in range(epochs):
    for signals, labels in training_set:
        tr_loss, tr_accuracy = model.train_on_batch(signals, labels)
    val_loss, val_accuracy = model.evaluate(X_data_val, y_labels_val)
    print(('Epoch #%d\t Training Loss: %.2f\tTraining Accuracy: %.2f\t'
         'Validation Loss: %.2f\tValidation Accuracy: %.2f')
         % (epoch + 1, tr_loss, tr_accuracy,
         val_loss, val_accuracy))

2021-07-14 14:12:49.548057: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-14 14:12:49.567821: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2500005000 Hz
8/8 [==============================] - 0s 1ms/step - loss: 1.4481 - accuracy: 0.3880
Epoch #1	 Training Loss: 1.43	Training Accuracy: 0.43	Validation Loss: 1.45	Validation Accuracy: 0.39
8/8 [==============================] - 0s 1ms/step - loss: 1.2369 - accuracy: 0.5080
Epoch #2	 Training Loss: 1.19	Training Accuracy: 0.51	Validation Loss: 1.24	Validation Accuracy: 0.51
8/8 [==============================] - 0s 1ms/step - loss: 1.1511 - accuracy: 0.5480
Epoch #3	 Training Loss: 1.14	Training Accuracy: 0.50	Validation Loss: 1.15	Validation Accuracy: 0.55
8/8 [==============================] - 0s 1ms/step - loss: 1.1085 - accuracy: 0.5280
Epoch #4	 Training Loss: 1.12	Training Accuracy: 0.49	Validation Loss: 1.11	Valida

## Check Performance on Test Set

We can use model.predict to output predicted labels on the test set, or model.evaluate to determine test-set accuracy (since we have the labels)


In [ ]:
test_loss, test_accuracy = model.evaluate(X_data_test, y_labels_test)

8/8 [==============================] - 0s 1ms/step - loss: 0.9599 - accuracy: 0.6120


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3d82bae0-f509-4ec0-92dc-a9b1bbc77ba9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>